<a href="https://colab.research.google.com/github/OseiasBeu/Cardiovascular-diseases-risk-project/blob/main/EDA_Cardiovascular_diseases_risk_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Características gerais do dataset:
- Registros: 308854;
- Features: 19;
- Features types:
  - Texto: 7;
  - Numérico: 12;
- Nulos: 0;
- Descrição das features:
  - Height_(cm): Altura em centímeros;
  - Weight_(kg): Peso em kilogramas;
  - BMI: Índice de massa corpórea (IMC)
  - Alcohol_Consumption: Consumo de àlcool
  -

# Kaggle configure

In [ ]:
# !pip install imbalanced-learn -q
# !pip install kaggle -q

In [ ]:

# from google.colab import files
# files.upload()
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets list
# %cd /content/
# !kaggle datasets download -d alphiree/cardiovascular-diseases-risk-prediction-dataset
# !unzip cardiovascular-diseases-risk-prediction-dataset.zip -d cardiovascular-diseases

# Lib Imports

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

In [ ]:
df_CVD = pd.read_csv('dataset/CVD_cleaned.csv')

# Explorando features

In [ ]:
df_CVD.head()

## Identificando Dtypes

In [ ]:
df_CVD.info()

* Quantidade de features: 19

In [ ]:
df_CVD.select_dtypes(include='object').info()

- Quantidade do tipo texto: 12

In [ ]:
df_CVD.select_dtypes(include='float').info()

- Quantidade do tipo numérico: 7

## Dtype exploration: Numéric

- Plotar a distribuição de cada variável numérica usando boxplots e histogramas;
- Testar estatisticamente cada variável numérica quanto à normalidade;
- Midir a correlação de cada variável numérica com outras usando um mapa de calor;
- Medir estatisticamente a inclinação e a curtose de cada variável.

In [ ]:
df_CVD.select_dtypes(include='float').describe().T

- Já conseguimos perceber alguns outliers que vamos precisar eliminar

### Outliers
- Plotar a distribuição de cada variável numérica usando boxplots e histogramas;

In [ ]:
numeric_data = df_CVD.select_dtypes(include = [np.number])
categorical_data = df_CVD.select_dtypes(exclude = [np.number]) # Selects categorical datatypes only

extreme_check_boxplot = sns.boxplot(data = df_CVD,
                                   orient = 'h')
plt.title('Distribuição de dados numéricos',
         fontweight = 'bold')
plt.show()

### Teste de normalidade
- Testar estatisticamente cada variável numérica quanto à normalidade;

Um teste de normalidade é um procedimento estatístico usado para verificar se uma amostra de dados segue uma distribuição normal (gaussiana). A distribuição normal é uma distribuição estatística importante que possui várias propriedades bem conhecidas e é frequentemente assumida por muitas técnicas estatísticas. No entanto, nem todos os conjuntos de dados seguem uma distribuição normal, e é importante verificar essa suposição antes de aplicar certas análises estatísticas.


A biblioteca scipy.stats no Python fornece uma função chamada normaltest que pode ser usada para realizar um teste de normalidade baseado em estatísticas de assimetria e curtose. O teste de normalidade é geralmente baseado na hipótese nula de que os dados vêm de uma distribuição normal. A função normaltest retorna duas estatísticas:

- A estatística de assimetria (skewness): Mede o grau de assimetria nos dados.
- A estatística de curtose (kurtosis): Mede o grau de achatamento da distribuição em relação à distribuição normal.

Além disso, a função também retorna um valor-p associado a essas estatísticas, que pode ser usado para determinar se os dados são estatisticamente significativamente diferentes de uma distribuição normal.

   * Se o valor-p for menor do que o nível de significância escolhido (geralmente 0.05), podemos rejeitar a hipótese nula de que os dados são normalmente distribuídos e concluir que os dados não seguem uma distribuição normal. Caso contrário, não temos evidências suficientes para rejeitar a hipótese nula e podemos assumir que os dados são normalmente distribuídos.

In [ ]:
from scipy.stats import normaltest
alpha = 0.05

for column in numeric_data:
    print(f'\t >{column}<')
    print(f'{column} Teste Estatistico: {normaltest(numeric_data[column])[0]}')
    print(f'{column} p-value: {normaltest(numeric_data[column])[1]}')

    if normaltest(numeric_data[column])[1] < alpha:
        print("Os dados não seguem uma distribuição normal.")
    else:
        print("Os dados seguem uma distribuição normal.")

    print(10*'-===-')


numerical_column_names = list(numeric_data.columns)

### Correlações:
- Midir a correlação de cada variável numérica com outras usando um mapa de calor;


As correlações de Pearson, Kendall's Tau e Spearman's Rho são três medidas estatísticas diferentes usadas para avaliar a relação entre duas variáveis. Elas são especialmente úteis para quantificar o grau e a direção da associação entre variáveis quantitativas ou ordinais. Cada uma dessas medidas tem suas próprias características e é apropriada para diferentes tipos de dados e situações. Vou explicar cada uma delas:

Correlação de Pearson (Coeficiente de Correlação de Pearson):
  A correlação de Pearson mede a relação linear entre duas variáveis contínuas. Ela varia de -1 a 1. Um valor de 1 indica uma correlação positiva perfeita, ou seja, as duas variáveis aumentam juntas em uma relação linear. Um valor de -1 indica uma correlação negativa perfeita, ou seja, uma variável aumenta enquanto a outra diminui linearmente. Um valor próximo de 0 indica uma correlação fraca ou ausência de correlação linear.
  No entanto, a correlação de Pearson assume que as variáveis têm uma relação linear e são distribuídas normalmente. Portanto, pode não ser apropriada para dados não lineares ou distribuições não normais.

Kendall's Tau (Tau de Kendall):
  O coeficiente de Kendall's Tau mede a associação de ordem entre duas variáveis. Ele é adequado para dados ordinais, ou seja, quando as observações podem ser classificadas em uma ordem específica, mas as diferenças entre as categorias não são necessariamente iguais. O coeficiente varia de -1 a 1, com valores mais próximos de -1 indicando uma associação negativa, valores mais próximos de 1 indicando uma associação positiva e valores próximos de 0 indicando ausência de associação.
  O Tau de Kendall é menos sensível a outliers do que a correlação de Pearson e não pressupõe uma relação linear entre as variáveis.

Spearman's Rho (Coeficiente de Correlação de Spearman):
  O coeficiente de Spearman mede a relação monotônica entre duas variáveis, o que significa que avalia se as variáveis têm uma relação que aumenta ou diminui em conjunto, mas não necessariamente de maneira linear. Assim como o Tau de Kendall, o Spearman's Rho é apropriado para dados ordinais e também pode ser usado com dados intervalares ou de razão.
  O coeficiente de Spearman também varia de -1 a 1, com interpretações semelhantes às outras medidas.

Resumindo:

- Pearson: Para dados contínuos, assume relação linear e normalidade.
- Kendall's Tau: Para dados ordinais, não assume uma relação linear.
- Spearman's Rho: Para dados ordinais ou intervalares, não assume uma relação linear.

A escolha entre essas medidas depende da natureza dos dados e da relação que se deseja avaliar. Cada medida tem seus próprios pontos fortes e limitações.

In [ ]:
numeric_spearman = pd.DataFrame(numeric_data).corr(method = 'spearman')
numeric_kendall = pd.DataFrame(numeric_data).corr(method = 'kendall')
numeric_pearson = pd.DataFrame(numeric_data).corr(method = 'pearson')

fig, ax = plt.subplots(nrows = 1,
                       ncols = 3,
                       sharex = True,
                       sharey = True)
plt.suptitle('Correlação entre as variáveis numéricas')

fig.set_size_inches(13, 5)

s_hmap = sns.heatmap(data = numeric_spearman, ax = ax[0], annot=True, fmt=".1f")
ax[0].set_title('Spearmans')

k_hmap = sns.heatmap(data = numeric_kendall, ax = ax[1], annot=True, fmt=".1f")
ax[1].set_title('Kendalls')

p_hmap = sns.heatmap(data = numeric_pearson, ax = ax[2],annot=True, fmt=".1f")
ax[2].set_title('Pearsons')


plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 4,
                      ncols = 2,
                      sharex = False,
                      sharey = True)
fig.set_size_inches(12, 10)

plt.suptitle('Histogramas para variáveis numéricas', fontweight = 'bold')

#Height
sns.histplot(data = df_CVD['Height_(cm)'], ax = ax[0, 0], bins = 30)
ax[0, 0].set_title('Height')
ax[0, 0].set_xlabel('Height (cm)')

#Weight
sns.histplot(data = df_CVD['Weight_(kg)'], ax = ax[0, 1], bins = 30)
ax[0, 1].set_title('Weight')
ax[0, 1].set_xlabel('Weight (kg)')

#BMI
sns.histplot(data = df_CVD['BMI'], ax = ax[1, 0], bins = 30)
ax[1, 0].set_title('BMI')
ax[1, 0].set_xlabel('BMI')

#Alcohol consumption
sns.histplot(data = df_CVD['Alcohol_Consumption'], ax = ax[1, 1], bins = 30)
ax[1, 1].set_title('Alcohol Consumption')
ax[1, 1].set_xlabel('Consumption')

#Fruit consumption
sns.histplot(data = df_CVD['Fruit_Consumption'], ax = ax[2, 0], bins = 30)
ax[2, 0].set_title('Fruit consumption')
ax[2, 0].set_xlabel('Consumption')

#Green vegetable consumption
sns.histplot(data = df_CVD['Green_Vegetables_Consumption'], ax = ax[2, 1], bins = 30)
ax[2, 1].set_title('Green Vegetable Consumption')
ax[2, 1].set_xlabel('Consumption')

#Fried Potato consumption
sns.histplot(data = df_CVD['FriedPotato_Consumption'], ax = ax[3, 0], bins = 30)
ax[3, 0].set_title('Fried Potato Consumption')
ax[3, 0].set_xlabel('Consumption')

ax[3, 1].set_axis_off()

plt.tight_layout()

### Verificando inclinação de kurtosis.
- Medir estatisticamente a inclinação e a curtose de cada variável.

A assimetria dos dados, também conhecida como assimetria estatística, é uma medida estatística que descreve a falta de simetria em uma distribuição de valores. Em outras palavras, a assimetria indica se a distribuição dos valores está inclinada para um lado em relação ao seu centro.

Uma distribuição simétrica é aquela em que os valores se distribuem de maneira uniforme em torno de um ponto central, como a média, a mediana e a moda. Porém, muitas vezes, os conjuntos de dados não apresentam essa simetria perfeita.

Existem três tipos principais de assimetria:

  * Assimetria Positiva (Skewness Positivo): Também chamada de assimetria para a direita, ocorre quando a cauda direita da * distribuição (valores mais altos) é mais longa do que a cauda esquerda. Isso significa que há valores extremamente altos que estendem a distribuição nessa direção.

  * Assimetria Negativa (Skewness Negativo): Também conhecida como assimetria para a esquerda, acontece quando a cauda esquerda da distribuição (valores mais baixos) é mais longa do que a cauda direita. Isso implica que há valores excepcionalmente baixos que puxam a distribuição nessa direção.

  * Assimetria Zero: Nesse caso, a distribuição é aproximadamente simétrica, com as caudas esquerda e direita tendo comprimentos semelhantes.

A medida mais comum para avaliar a assimetria é o coeficiente de assimetria de Pearson, ou simplesmente coeficiente de assimetria. Essa medida utiliza a média, a mediana e o desvio padrão para calcular a magnitude e a direção da assimetria. Se o coeficiente de assimetria for próximo de 0, a distribuição é aproximadamente simétrica. Se for positivo, há assimetria positiva, e se for negativo, há assimetria negativa.

A assimetria é importante porque pode afetar a interpretação dos dados e a escolha das técnicas estatísticas apropriadas. Em análises estatísticas, especialmente aquelas que pressupõem normalidade, é essencial considerar a assimetria dos dados para tomar decisões informadas sobre quais métodos e abordagens utilizar.

A estatística de curtose (kurtosis) é uma medida estatística que descreve o achatamento ou a "cauda" de uma distribuição de valores em relação à distribuição normal. Em outras palavras, a curtose mede a concentração dos valores em torno da média em comparação com a distribuição normal.

Uma curtose alta indica que os valores da distribuição têm uma concentração maior ao redor da média, resultando em caudas mais pesadas (outliers mais extremos). Uma curtose baixa indica uma concentração menor em torno da média, com caudas mais leves.

Existem duas medidas comuns de curtose:

Curtose de Pearson: Essa medida é definida como a média dos desvios à quarta potência dos valores em relação à média da distribuição. Ela é normalmente comparada à curtose da distribuição normal, que é 3. Portanto, a curtose de Pearson é frequentemente relatada em relação a 3. Uma curtose maior que 3 indica uma distribuição mais concentrada, com caudas mais pesadas, enquanto uma curtose menor que 3 indica uma distribuição menos concentrada, com caudas mais leves.

Curtose Excessiva: Para facilitar a interpretação, também é comum calcular o "excesso de curtose", que é simplesmente a curtose de Pearson menos 3. Isso torna a curtose da distribuição normal igual a 0, o que facilita a comparação.

A interpretação da curtose depende do contexto e dos dados envolvidos:

Curtose positiva (maior que 3 ou excesso de curtose positivo): Indica uma distribuição mais concentrada e com caudas mais pesadas do que a distribuição normal. Isso pode indicar a presença de valores extremos (outliers) na distribuição.

Curtose negativa (menor que 3 ou excesso de curtose negativo): Indica uma distribuição menos concentrada e com caudas mais leves do que a distribuição normal. Isso pode sugerir uma distribuição mais uniforme dos valores.

Lembre-se de que a interpretação da curtose deve ser feita em conjunto com outras análises e contextos específicos dos dados. A curtose não é uma medida de normalidade, e uma distribuição com curtose diferente da normal não necessariamente é inadequada para análises estatísticas. Em alguns casos, distribuições não normais podem ser mais apropriadas para representar os dados reais.

In [ ]:
from scipy.stats import skew, kurtosis

print('Assimetria da distribuição de dados das variáveis numéricas')
print('------------------------------')
print()

for column in numeric_data:
    print(f'\t{column}')
    print(f'{column} skew: {skew(numeric_data[column])}')
    print(f'{column} kurtosis: {kurtosis(numeric_data[column])}')
    print()

A estatística de curtose (kurtosis) é uma medida estatística que fornece informações sobre a forma da distribuição de um conjunto de dados, especificamente em relação ao pico central e às caudas da distribuição. Ela está relacionada à concentração dos valores em torno da média e à extensão das caudas da distribuição.

A curtose nos diz como os valores se comportam nas regiões mais extremas da distribuição em comparação com a distribuição normal. Existem dois aspectos principais que a curtose pode revelar:

* 1. Achatamento (Leptocurtose) ou Alargamento (Platicurtose): A curtose pode indicar se os valores da distribuição estão concentrados em torno da média (achatamento) ou espalhados (alargamento). Uma curtose alta (positiva) sugere que a distribuição é mais concentrada em torno da média e possui caudas mais pesadas, o que significa que há uma maior incidência de valores extremos. Uma curtose baixa (negativa) indica que a distribuição é mais achatada em relação à distribuição normal, com caudas mais leves.

* 2. Presença de Outliers: Uma curtose alta pode indicar a presença de valores extremos (outliers) na distribuição. Isso ocorre porque esses valores extremos contribuem para as caudas mais pesadas da distribuição.

Geralmente, a curtose é comparada à curtose da distribuição normal, que é igual a 3. Se a curtose de uma distribuição for maior do que 3, dizemos que ela tem uma curtose excessiva (leptocurtose), o que significa que tem caudas mais pesadas e é mais concentrada em torno da média. Se a curtose for menor do que 3, dizemos que ela tem uma curtose insuficiente (platicurtose), o que significa que tem caudas mais leves e é mais achatada.

No entanto, é importante ter em mente que a interpretação da curtose deve ser feita com cautela. A curtose não é uma medida de normalidade e não indica necessariamente que uma distribuição seja normal ou não. Além disso, a interpretação da curtose pode variar dependendo do contexto dos dados e da análise estatística que está sendo realizada. Em alguns casos, uma distribuição não normal ou com curtose diferente da normal pode ser apropriada para representar os dados reais. Portanto, é sempre aconselhável considerar a curtose em conjunto com outras análises estatísticas e contexto dos dados.

## Dtype exploration: Text

 - Listar as informações básicas sobre cada feature categórica
 - Correção da ordem e nomes das variáveis, se necessário
 - Plotar histogramas de cada variável categórica

In [ ]:
# categorical_column_names

In [ ]:
categorical_column_names = list(df_CVD.columns)

print(f'Lista de variáveis categóricas: {list(categorical_data)}')
print(f'Número de variáveis categóricas: {len(list(categorical_data))}')
print()
print()
print('-----------------------------------------------------------------------')

for col in categorical_data:
    print(f'{col} Valores distintos: {list(categorical_data[col].unique())}')
    print(f'{col} tem {len(list(categorical_data[col].unique()))} valores únicos')
    print()
    print()

In [ ]:
df_CVD['Checkup'] = df_CVD['Checkup'].replace({'Within the past 2 years': '<2 anos',
                                                  'Within the past year': '<1 ano',
                                                  '5 or more years ago': '5+ anos',
                                                  'Within the past 5 years': '<5 anos',
                                                  'Never': 'Nunca'})

In [ ]:
fig, ax = plt.subplots(nrows = 3,
                      ncols = 3,
                      sharex = False,
                      sharey = True)
fig.set_size_inches(12, 10)

plt.suptitle('Contagem de todas as variáveis categóricas',
            fontweight = 'bold')

#General Health
sns.histplot(data = df_CVD['General_Health'], stat = 'count', ax = ax[0, 0])
ax[0, 0].set_title('Health')
ax[0, 0].set_xlabel('Categorias de saúde')


#exercise
sns.histplot(data = df_CVD['Exercise'], stat = 'count', ax = ax[0, 1])
ax[0, 1].set_title('Exercise')
ax[0, 1].set_xlabel('Grupo de exercícios')


#Heart Disease
sns.histplot(data = df_CVD['Heart_Disease'], stat = 'count', ax = ax[0, 2])
ax[0, 2].set_title('Heart Disease')
ax[0, 2].set_xlabel('Grupos de doenças de saúde')

#Skin Cancer
sns.histplot(data = df_CVD['Skin_Cancer'], stat = 'count', ax = ax[1, 0])
ax[1, 0].set_title('Skin Cancer')
ax[1, 0].set_xlabel('Grupos de canceres de pele')

#Other cancer
sns.histplot(data = df_CVD['Other_Cancer'], stat = 'count', ax = ax[1, 1])
ax[1, 1].set_title('Other cancers')
ax[1, 1].set_xlabel('Outros grupos de cancer')

#Depression
sns.histplot(data = df_CVD['Depression'], stat = 'count', ax = ax[1, 2])
ax[1, 2].set_title('Depression')
ax[1, 2].set_xlabel('Grupos de depressao')


#Arthritis
sns.histplot(data = df_CVD['Arthritis'], stat = 'count', ax = ax[2, 0])
ax[2, 0].set_title('Arthritis')
ax[2, 0].set_xlabel('Grupo de artrite')

#Sex
sns.histplot(data = df_CVD['Sex'], stat = 'count', ax = ax[2, 1])
ax[2, 1].set_title('Sex')
ax[2, 1].set_xlabel('Sexo')

#Smoking History
sns.histplot(data = df_CVD['Smoking_History'], stat = 'count', ax = ax[2, 2])
ax[2, 2].set_title('Smoking')
ax[2, 2].set_xlabel('Grupo de histórico de fumantes')


plt.tight_layout()
plt.show()

In [ ]:
sns.histplot(data = df_CVD['Checkup'], stat = 'count')
plt.xticks(rotation = 30)
plt.title('Checkup histogram')

In [ ]:
sns.histplot(data = df_CVD['Diabetes'], stat = 'count')
plt.xticks(rotation = 30)
plt.title('Diabetes histogram')

In [ ]:
sns.histplot(data = df_CVD['Age_Category'], stat = 'count')
plt.title('Age group histogram')
plt.xticks(rotation = 30)

In [ ]:
def compare_categoricals(data, column_names_list, comparison_col = None): # comparison_col is optional - include if want multicategorical histograms
    # print(data.columns)
    # print(column_names_list)
    fig, ax = plt.subplots(nrows = 11,
                          ncols = 1,
                          sharex = False,
                          sharey = False)

    fig.set_size_inches(14, 26)

    plt.suptitle(f'Colunas categóricas por: {comparison_col}', y = 0.99)
    plt.subplots_adjust(hspace = 1)

    i = 0

    for column in column_names_list:

      if str(column) == str(comparison_col):
        pass # Skip and don't plot anything if its the same.
      elif i == 11:
        break
      else:
        sns.histplot(data = data, x = column, hue = comparison_col, ax = ax[i], multiple ='dodge')
        ax[i].set_title(f'{str(column)} and {str(comparison_col)}')
        ax[i].set_ylabel('Count')

      i = i + 1 # New row for next categorical variable


    plt.tight_layout()
    plt.show()

In [ ]:
compare_categoricals(df_CVD[categorical_column_names], categorical_column_names, 'Heart_Disease')

## Analisando variáveis categóricas-numéricas

Criar uma função que:

- Tenha o input de variáveis categóricas e variáveis numéricas e exiba uma comparação entre ambos os tipos
- Plote gráficas de violino e ecdf de cada combinação de variáveis categóricas e númericas

In [ ]:
def univariate_categorical_by_numerical(data, numerical_cols, categorical_col = None):

    fig, ax = plt.subplots(nrows = 7,
                          ncols = 2,
                          sharex = False,
                          sharey = False)
    fig.set_size_inches(16, 20)

    i = 0 # Y

    for col in numerical_cols:

        sns.violinplot(data = data, x = categorical_col, y = col, ax = ax[i, 0])
        ax[i, 0].set_title(f'{str(col)} distribuido por {str(categorical_col)}')
        ax[i, 0].set_xlabel(f'{categorical_col}')

        sns.ecdfplot(data = data, hue = categorical_col, x = col, ax = ax[i, 1])
        ax[i, 1].axhline(0.5, linestyle = 'dashed')

        i += 1

    plt.tight_layout()
    plt.show()

Um gráfico de ECDF (Empirical Cumulative Distribution Function) fornece uma maneira simples de entender como os valores em um conjunto de dados estão distribuídos. Ele mostra quanto percentual dos valores são menores ou iguais a um determinado valor.

Em resumo, um gráfico de ECDF permite que você tenha uma ideia rápida de como os valores estão espalhados em um conjunto de dados. É uma maneira poderosa de ver a distribuição de seus dados sem precisar fazer suposições sobre o formato da distribuição.

In [ ]:
univariate_categorical_by_numerical(df_CVD, numerical_column_names, 'Heart_Disease')

## Analises multinuméricas

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
#Multi-numeric scatterplot
def plot_multi_numerics_scatter(data, numerical_cols, comparison_col, categorical_col = None): # Optional Categorical column

    figure = make_subplots(rows = 2, cols = 3) # 6 plots, as won't plot against self.

    #Hardcoded
    row_pos = [1, 1, 1, 2, 2, 2]
    row_pos_count = 0

    col_pos = [1, 2, 3, 1, 2, 3]
    col_pos_count = 0

    for col in numerical_cols:

        if col != comparison_col:


            scatter = px.scatter(data_frame = data, x = comparison_col, y = col, color = categorical_col, opacity = 0.35, trendline = 'ols', trendline_scope = 'overall')
            for i in scatter.data:
                if row_pos_count != 1 and col_pos_count != 1: # No legend for all plots except 1.
                    i.update(legendgroup = 'group', showlegend = False)
                    figure.add_trace(i, row = row_pos[row_pos_count], col = col_pos[col_pos_count])
                    figure.update_xaxes(title_text = str(comparison_col), row = row_pos[row_pos_count], col = col_pos[col_pos_count])
                    figure.update_yaxes(title_text = str(col), row = row_pos[row_pos_count], col = col_pos[col_pos_count])

                else:
                    i.update(legendgroup = 'group', showlegend = True) # Show the legend for just the top left plot
                    figure.add_trace(i, row = row_pos[row_pos_count], col = col_pos[col_pos_count])
                    figure.update_xaxes(title_text = str(comparison_col), row = row_pos[row_pos_count], col = col_pos[col_pos_count])
                    figure.update_yaxes(title_text = str(col), row = row_pos[row_pos_count], col = col_pos[col_pos_count])


            row_pos_count = row_pos_count + 1
            col_pos_count = col_pos_count + 1

        else: #Do this if the numerical col is the same as the comparison col.
            pass


    figure.update_layout(title_text = 'Comparação de variáveis numéricas', legend_title_text = str(categorical_col))
    figure.show()

In [ ]:
plot_multi_numerics_scatter(df_CVD, numerical_column_names, 'Weight_(kg)', 'Heart_Disease')

In [ ]:
#Multi-numeric density heatmap. Optional filter to a specific grouping by categorical variable.
def plot_multi_numerics_density_heatmap(data, numerical_cols, comparison_col, xbins, ybins, categorical_col = None, categorical_filter = None):

    figure = make_subplots(rows = 2, cols = 3) # 6 plots, as won't plot against self.

    if categorical_col != None:
        data = data[data[categorical_col] == categorical_filter]
    else:
        pass

    #Hardcoded
    row_pos = [1, 1, 1, 2, 2, 2]
    row_pos_count = 0

    col_pos = [1, 2, 3, 1, 2, 3]
    col_pos_count = 0

    for col in numerical_cols:

        if col != comparison_col:


            density = px.density_heatmap(data_frame = data, x = comparison_col, y = col, nbinsx = xbins, nbinsy = ybins)
            for i in density.data:
                if row_pos_count != 1 and col_pos_count != 1: # No legend for all plots except 1.
                    i.update(legendgroup = 'group', showlegend = False)
                    figure.add_trace(i, row = row_pos[row_pos_count], col = col_pos[col_pos_count])
                    #figure.update_traces(contours_coloring = 'fill')
                    figure.update_xaxes(title_text = str(comparison_col), row = row_pos[row_pos_count], col = col_pos[col_pos_count])
                    figure.update_yaxes(title_text = str(col), row = row_pos[row_pos_count], col = col_pos[col_pos_count])

                else:
                    i.update(legendgroup = 'group', showlegend = True) # Show the legend for just the top left plot
                    figure.add_trace(i, row = row_pos[row_pos_count], col = col_pos[col_pos_count])
                    #figure.update_traces(contours_coloring = 'fill')
                    figure.update_xaxes(title_text = str(comparison_col), row = row_pos[row_pos_count], col = col_pos[col_pos_count])
                    figure.update_yaxes(title_text = str(col), row = row_pos[row_pos_count], col = col_pos[col_pos_count])


            row_pos_count = row_pos_count + 1
            col_pos_count = col_pos_count + 1

        else: #Do this if the numerical col is the same as the comparison col.
            pass

    if categorical_col != None and categorical_filter != None:

        figure.update_layout(title_text = f'Mapa de calor da densidade das variáveis numéricas - {xbins} x {ybins} bins : {categorical_col} = {categorical_filter}', autosize = False, width = 1050, height = 1050)
    else:
        figure.update_layout(title_text = f'Mapa de calor da densidade das variáveis numéricas - {xbins} x {ybins} bins', autosize = False, width = 1050, height = 1050)



    figure.show()

In [ ]:
plot_multi_numerics_density_heatmap(df_CVD, numerical_column_names, 'Weight_(kg)', 50, 50,'Heart_Disease', 'Yes')

In [ ]:
plot_multi_numerics_density_heatmap(df_CVD, numerical_column_names, 'Weight_(kg)', 50, 50,'Heart_Disease', 'No')

# Principais insights

- A maioria dos indivíduos não tinha sido diagnosticada com doença cardíaca;
- A doença cardíaca foi ligeiramente mais prevalente entre os indivíduos do sexo masculino;
- Indivíduos diagnosticados com doenças cardíacas tendiam a consumir mais álcool, ter mais peso, comer menos frutas, tinham histórico de tabagismo e eram mais velhos;

In [ ]:
df_CVD.head(5)

In [ ]:
df_CVD.info()